# Planetary Training

OBS: This notebook is for running in the MSPC only!!!!

In [1]:
%load_ext autoreload
%autoreload 2

import importlib    

# check if it is using CPU or GPU
# First the CPU version
if importlib.util.find_spec('torch') is None:
    %pip install --quiet torch torchvision --extra-index-url https://download.pytorch.org/whl/cpu
    
    # Then install fastai from code
    !cd /tmp; git clone https://github.com/fastai/fastai -q
    %pip -q install -e /tmp/fastai    

# Install WaterNet
if importlib.util.find_spec('waternet') is None:
    %pip install -e /home/jovyan/projects/WaterNet/
    
from waternet.wndataset import WNDataSet
from fastai.vision.all import *



### Creating the Datasets

In [2]:
masks_path = '/home/jovyan/tmp3'
training_bands = ['B02', 'B03', 'B04', 'B08', 'B11', 'B12']
# masks_path='d:/temp/22KEV/'

In [3]:
# default config for shape, size and step
config = dict(
    shape=(5490, 5490),
    patch_size=(512, 512),
    step=262,    
)

# 2018 october for training
train_ds = WNDataSet.from_masks(
    masks_path=masks_path,
    pattern='*S2A*watermask.tif',
    **config
)

Matching imgs/masks:   0%|          | 0/106 [00:00<?, ?it/s]

In [4]:
# Testing a workload...
# from tqdm.notebook import tqdm
# from time import sleep 

# train_ds.bands = training_bands
# for patch, mask in tqdm(train_ds):
#     patch.mean()
#     sleep(0.01)

In [5]:
# a single image for the validation
valid_ds = WNDataSet.from_masks(
    masks_path=masks_path,
    pattern='*S2B_MSIL2A_2021*watermask.tif',
    **config
)

Matching imgs/masks:   0%|          | 0/25 [00:00<?, ?it/s]

In [6]:
train_ds.bands = training_bands
valid_ds.bands = training_bands

print(train_ds)
print(valid_ds)

WNDataset instance with 106 images
Loaded: 0 items
Empty: 106 items
Loading: 0 items

WNDataset instance with 25 images
Loaded: 0 items
Empty: 25 items
Loading: 0 items



### Creating Dataloaders

In [7]:
# Create two dataloaders (for training and validation)
# Put it in the GPU
dls = DataLoaders(
    DataLoader(train_ds, bs=8),
    DataLoader(valid_ds, bs=8)
) # .cuda()

In [32]:
# m, t = dls.valid.one_batch()
# m.device

In [33]:
# dls.train.one_batch()[0].shape

### Creating the Learner

In [8]:
learner = unet_learner(
    dls, arch=resnet18, pretrained=True, normalize=False, n_in=6, n_out=3, loss_func=CrossEntropyLossFlat(axis=1)
)
# cuda_model = learner.cuda()

Loading image 1 in background


/srv/conda/envs/notebook/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loaded sucessfully


### Checking model and predictions

In [17]:
learner.unfreeze()

In [18]:
learner.summary()

DynamicUnet (Input shape: 8 x 6 x 512 x 512)
Layer (type)         Output Shape         Param #    Trainable 
                     8 x 64 x 256 x 256  
Conv2d                                    18816      True      
BatchNorm2d                               128        True      
ReLU                                                           
____________________________________________________________________________
                     8 x 64 x 128 x 128  
MaxPool2d                                                      
Conv2d                                    36864      True      
BatchNorm2d                               128        True      
ReLU                                                           
Conv2d                                    36864      True      
BatchNorm2d                               128        True      
Conv2d                                    36864      True      
BatchNorm2d                               128        True      
ReLU                      

In [10]:
batch, target = dls.train.one_batch()
print(batch.shape, target.shape)

torch.Size([8, 6, 512, 512]) torch.Size([8, 512, 512])


In [11]:
# Calculate the predictions for the 4 patches (1 batch)
pred = learner.model(batch)
pred.shape

torch.Size([8, 3, 512, 512])

In [12]:
CrossEntropyLossFlat(axis=1)(pred, target)

TensorBase(0.9229, grad_fn=<AliasBackward0>)

### Running LrFind

Up to now, evertying was done in the CPU, but to run large amounts, it is necessary to load the data in the Cuda device. For that we will put them in the GPU and find the best learning rate.

In [13]:
learner.lr_find()

KeyboardInterrupt: 

In [16]:
learner.opt_func?

Signature:
learner.opt_func(
    params,
    lr,
    mom=0.9,
    sqr_mom=0.99,
    eps=1e-05,
    wd=0.01,
    decouple_wd=True,
)
Docstring: A `Optimizer` for Adam with `lr`, `mom`, `sqr_mom`, `eps` and `params`
File:      /tmp/fastai/fastai/optimizer.py
Type:      function


In [ ]:
learner.fit_one_cycle(1, lr_max=1e-4)

In [ ]:
learner.model_dir

In [ ]:
learner.save('Resnet18-6chnls-untrained')

### Como ver os resultados???

In [ ]:
from waternet.wnvisualizer import WNVisualizer

In [ ]:
learner = unet_learner(
    dls, arch=resnet18, pretrained=False, normalize=False, n_in=6, n_out=3, loss_func=CrossEntropyLossFlat(axis=1)
)
learner.load('Resnet18-6chnls-untrained')
model = learner.cpu()

In [ ]:
visu = WNVisualizer(dls.valid, learner)

In [ ]:
visu.predict_item(0)['item'].shape

In [ ]:
visu.show_pred(800)

In [ ]:
dls.valid.dataset.loaded_status